# Introduction to Deep Learning
## Theory and Practice with TensorFlow and Keras
<img src="http://i.imgur.com/Gk8rv2Z.jpg" width=700>
https://arxiv.org/abs/1508.06576<br/>
*by the end of this course, this paper and project will be accessible to you!*


### Schedule

* Intro
* TensorFlow Basics
* Artificial Neural Networks
* Multilayer ("Deep") Feed-Forward Networks
* Training Neural Nets
* Convolutional Networks
* Recurrent Nets, LSTM, GRU
* Generative Networks / Patterns
* Intro to Reinforcement Learning
* Operations in the Real World

### Instructor: Adam Breindel

<img src="http://i.imgur.com/lpMDU9j.jpg" width=200 align=left style="margin-right:2em;margin-top:1em">

#### Contact: https://www.linkedin.com/in/adbreind - adbreind@gmail.com

* Almost 20 years building systems for startups and large enterprises
* 10 years teaching front- and back-end technology

#### Interesting projects...
* My first full-time job in tech involved streaming neural net fraud scoring (debit cards)
* Realtime & offline analytics for banking
* Music synchronization and licensing for networked jukeboxes

#### Industries
* Finance / Insurance, Travel, Media / Entertainment, Government

### Class Goals
* Understand deep learning!
    * Acquire an intiution and feeling for how and why and when it works, so you can use it!
    * No magic! (or at least very little magic)
    
* We *don't* want to have a workshop where we install and demo some magical, fairly complicated thing, and we watch it do something awesome, and handwave, and go home
    * That's great for generating excitement, but leaves
        * Theoretical mysteries -- what's going on? do I need a Ph.D. in Math or Statistics to do this?
        * Practical problems -- I have 10 lines of code but they never run because my tensor is the wrong shape!
        
* We'll focus on TensorFlow and Keras 
    * But 95% should be knowledge you can use with frameworks too: Intel BigDL, Baidu PaddlePaddle, NVIDIA Digits, MXNet, etc.

## Deep Learning is About Machines Finding Patterns

Actually, all machine learning as about machines finding patterns...

Some learning approaches constrain those patterns more, so the computer can discover less,<br/>
...but can do so faster and more reliably -- "higher bias, lower capacity" models.

### Deep learning has been around for a long time...<br/>but has broken through because of its success handling "95%" problems


__What are "95%" problems? Problems where 95% sounds like a great success rate, but in reality is a disaster__

Examples: speech reconition, handwriting recognition, image recognition, control systems and automation

__So why focus on deep learning? Instead of arguing that further now, let's learn about it and revisit this question at the end of class__

At that point, you'll have a really good handle on whether and why to focus on deep learning in your projects!

# Let's Start By Looking at a Problem

<img src="http://i.imgur.com/uggRlE7.png" width=600>

## MNIST Digits Dataset
### Mixed National Institute of Standards and Technology
#### Called the "Drosophila" of Machine Learning

Likely the most common single dataset out there in deep learning, just complex enough to be interesting and useful for benchmarks. "If your code works on MNIST, that doesn't mean it will work everywhere, but if it doesn't work on MNIST, it probably won't work anywhere" :)

### What is the goal?

Convert an image of a handwritten character into the correct classification (i.e., which character is it?)

This is nearly trivial for a human to do! Most toddlers can do this with near 100% accuracy, even though they may not be able to count beyond 10 or perform addition.

Traditionally this had been quite a hard task for a computer to do. 99% was not achieved until ~1998. Consistent, easy success at that level was not until 2003 or so.

### How might we go about solving this, in general?

* How do we get information out of the images?
    * each image is 28x28 pixels -- originally monochrome; smoothed to gray; typically inverted, so that "blank" pixels are black (zeros)
    
* Each digit is a 784-vector
    * Clearly, not all pixels are equally important ...
    * Should we downsample?
    * Test every pixel for significance against the target (e.g., Chi Square selection)?
    * Reduce dimensionality via PCA?
    * Write a procedure to discover "important" features like lines, loops, and hooks, and then write a procedure that classifies based on those higher-order features? if so, how?


### All of these are possible approaches! 

Let's explore one straightforward method: we'll build a ...

### Decision Tree over the Pixels

Basic approach
* "CART" or "Classification and Regression Tree" (https://en.wikipedia.org/wiki/Decision_tree_learning)
* Iteratively choose a "split condition" that maximizes information gain
    * i.e., intuitively, the split that gets us "closest" to a correct categorization
* Parameters include
    * Max depth
    * Discretization of continuous values

Decision trees have some interesting properties compared to other learning algorithms
* They implicitly perform __feature selection__ 
    * Typically greedy, so features with the most predictive power bubble up higher in the tree
    * Features with lower predictive power may not be in the model at all
    * Given sufficient depth, an amenable data set, and a brute force training approach, they can perfectly fit the training data
        * which is usually a bug, not a feature
        * consider classifying 8x8-pixel monochrome images using a tree of depth 64
            * you'll get a leaf for every possible image, at which point you just have a lookup table of all the answers and a binary search to speed things up ... it's hard to argue that such a structure has learned anything
            
__Let's run some code just to make this real! Then we'll talk a bit more about the theoretical considerations__

In [ ]:
%%sh 

head -1 data/mnist

In [ ]:
import sklearn
import sklearn.datasets
from sklearn.tree import DecisionTreeClassifier

train_libsvm = "data/mnist"

X, y = sklearn.datasets.load_svmlight_file(train_libsvm, n_features=784)
dt = DecisionTreeClassifier(random_state=0, max_depth=2)
fittedModel = dt.fit(X, y)

print(fittedModel)

In [ ]:
import pydot
from IPython.display import Image

dot_data = sklearn.tree.export_graphviz(fittedModel, out_file=None)
graph = pydot.graph_from_dot_data(dot_data)
Image(graph[0].create_png(), height=500) 

In [ ]:
from sklearn.metrics import accuracy_score

test_libsvm = "data/mnist.t"

X_test, y_test = sklearn.datasets.load_svmlight_file(test_libsvm, n_features=784)

y_pred = fittedModel.predict(X_test)
print("Accuracy %f" % accuracy_score(y_test, y_pred) )

In [ ]:
dt = DecisionTreeClassifier(random_state=0, max_depth=10)
fittedModel = dt.fit(X, y)
y_pred = fittedModel.predict(X_test)
print("Accuracy %f" % accuracy_score(y_test, y_pred) )


### Important Questions
In this particular context, we can think about:
* Feature selection: which pixels are most predictive? how can we identify them?
* How many splits should we allow to balance generality with precision?
* How *reliable* is a set of splits? is there a statistical way we can talk about the discriminatory power of the model?
* What happens if we translate the images a little? rotate them a little? shear/skew?

__Why do we think this problem should be solvable? After all, lots of ML problems aren't, at least not to a high degree of accuracy?__

Because (1) we do it easily as humans so (2) we are convinced there is a more than enough *information* in the original images to solve the problem.

In fact, we have an excess of information in most cases, therefore we can discard lots of information and still get a decent result.

### What Might Be Some Other Ways of Approaching This?

#### Logistic Regression

A linear combination of parameters that gives the log-odds of a particular class outcome.

<img src="http://i.imgur.com/LBi5OEo.jpg" width=500>

If we simplify to 2 classes (say, 0 and 1 digits) this is easier to think about. Multiclass is just an extension of this approach.

This is interesting because we are fitting the probability (log-odds but similar idea) of the record matching a particular class.

How do we decide which class to assign to? Take the larger probability (or largest of n). But ... this opens up some intriguing possibilities, because when we make a mistake, we're not "just wrong" -- we know whether we were a lot or a little wrong ... and we can optimize that.

I.e., there's information in the prediction besides just true/false or accuracy. So we can correct small errors differently from large ones. This is a key principle that we will use in many deep learning scenarios, and it helps us get used to the idea that we are trying to learn distributions, not just absolute answers.

#### Support Vector Machines

SVMs, especially SVMs with RBF (radial-basis-function kernels) were the state of the art for MNIST prior to the advent of deep learning.

<img src="http://i.imgur.com/y7AGyUd.jpg" width=700>

### Probabilistic Scoring and Cross Entropy

Traditional measures like Accuracy, Precision, Recall, and F1 score are all based on counting right and wrong classification. And at the end of the day, as humans, they are often the numbers we need to make a decision, such as "How frequently will my classifier correctly identify fraud, or spam?"

But when we train our model, it's very helpful to know if we're "off by a little" or "off by a lot" and in what direction.

What we really want to know is how *surprised* are we by the mistake? How seriously do we need to fix it? We'll talk about the mathematical translation of that idea, but the idea is more important than the math.

And surprise -- how much signal or energy there is, or how unlikely a particular signal is to occur -- is the mathematical basis for quantifying information.

The goal we're getting to is a measure called Cross Entropy, but we want to fill in some intuition first.

As engineers -- or even users of computers -- we have this in our bones already from working with compression!

We perform encoding all the time based on this exact principle, so we just need to transfer this intuition to the general case:

### Information

* Information theory
* Ralph Hartley

$H = log S^n = n log S$

* Shannon *Entropy* in bits/symbol where $p_i$ is the probability of occurrence of the *i*-th possible value of the source symbol

${\displaystyle H=-\sum _{i}p_{i}\log _{2}(p_{i})}$


$\mathrm {H} (X)=\mathrm {E} [\mathrm {I} (X)]=\mathrm {E} [-\ln(\mathrm {P} (X))]$

${\displaystyle \mathrm {H} (X)=\sum _{i=1}^{n}{\mathrm {P} (x_{i})\,\mathrm {I} (x_{i})}=-\sum _{i=1}^{n}{\mathrm {P} (x_{i})\log _{b}\mathrm {P} (x_{i})}}$


### Measuring the Difference Between Distributions

The Kullback–Leibler divergence (or information gain or relative entropy) measures the non-symmetric difference between two probability distributions P and Q.

What does this mean? It's a measure of the information or surprise in a distribution P where we were expecting Q.

Thinking about it from a data-compression POV, if your data really comes from probability distribution P, but you use a compression scheme optimised for Q, the divergence D(P||Q) is the number of extra bits you'll need to encode each sample from P.

You don't need the formula, but it's not too crazy:
$$D_{\mathrm {KL} }(P\|Q)=\sum _{i}P(i)\,\log {\frac {P(i)}{Q(i)}}$$

(this is for discrete distributions; there's an equivalent one with integration for continuous distributions)

### Aside: a Little More Terminology

* "Prior" distribution - assumptions about the distribution underlying our samples. For example, let's say we were modeling Amazon review scores, we might initially assume a "uniform prior" (any star rating is equally likely). But in fact most Amazon reviews have 4 or 5 stars. Once we know that, we can incorporate this "prior" into our modeling.

* Frequentist probability - the probability of an event is the roughly the proportion of times it occurs out of many trials

* Bayesian probability - the probability of an event represents our belief in its likelihood given all of the information we know

The main theoretical distinction is that for a Bayesian, a single event can have a meaningful probability.

### Cross Entropy

Cross entropy is then defined as 

${\displaystyle H(p,q)=\operatorname {E} _{p}[-\log q]=H(p)+D_{\mathrm {KL} }(p\|q)\!}$

which for discrete p and q distributions is

${\displaystyle H(p,q)=-\sum _{x}p(x)\,\log q(x).\!}$

Here's a concrete example that will help. Consider the following two classifiers, which both get the same "total accuracy":

__Classifier 1__
```
computed       | targets              | correct?
-----------------------------------------------
0.3  0.3  0.4  | 0  0  1 (democrat)   | yes
0.3  0.4  0.3  | 0  1  0 (republican) | yes
0.1  0.2  0.7  | 1  0  0 (other)      | no
```
__Classifier 2__
```
computed       | targets              | correct?
-----------------------------------------------
0.1  0.2  0.7  | 0  0  1 (democrat)   | yes
0.1  0.7  0.2  | 0  1  0 (republican) | yes
0.3  0.4  0.3  | 1  0  0 (other)      | no
```

Classifier 2 is clearly doing much better at estimating class probabilities, but accuracy score doesn't tell us that.

Let's calculate the cross-entropy error for both classifiers (output distribution is q, real is p):

Over all the items, the average cross-entropy for Classifier 1 is `-(ln(0.4) + ln(0.4) + ln(0.1)) / 3 = 1.38`

whereas for Classifier 2 it's `-(ln(0.7) + ln(0.7) + ln(0.3)) / 3 = 0.64`

__The takeaway? When we care about precise direction and quantity of error -- which we will want to train our models -- cross-entropy will be more effective than accuracy__

### Wrap Up: Comparing Pros/Cons of Three Classification Models

#### Decision Tree

&#x1f44d; High "capacity" (high variance)

&#x1f44e; Can be hard to generalize; prone to overfit; does not create smooth space!

&#x1f44e; Dedicated training algorithm (traditional approach is not a convex optimization problem)

&#x1f44d; Performs features selection / PCA implicitly

&#x1f44d; Easy to understand, explain, alter final model

... let's try to visualize the spaces that decision trees carve out.

#### Logistic Regression

&#x1f44e; Low capacity/variance -> High bias

&#x1f44d; Less overfitting

&#x1f44e; Less fitting (it's a linear model)

&#x1f610; Human explanatory difficulty is intermediate

&#x1f44d; With the sigmoid, we get a smooth differentiable separation of spaces

<img src="http://i.imgur.com/1x80BDA.png" width=400>

#### Support Vector Machine

&#x1f44d; Robust capacity

&#x1f44e; Hard to explain the model

&#x1f44d; Cost function can be made differentiable

&#x1f44d; Amenable to "online" learning (i.e., record-at-a-time update)
